In [ ]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import numpy
import requests
import pymongo

In [ ]:
url = "https://docs.google.com/spreadsheets/d/1zLNAuRqPauss00HDz4XbTH2HqsCzMe0pR8QmD1K8jk8/pubhtml/sheet?headers=false&chrome=false&gid=0&range=A3:I437"

In [ ]:
df = pd.read_html(url)

In [ ]:
df = df[0]

In [ ]:
df.rename(inplace=True, columns={1:'District', 5:'2016', 7:'2012', 9:'2008'})

In [ ]:
df = df[["District", "2016", "2012", "2008"]]
df.head()

In [ ]:
def fixDistricts(district):
    state = district[0:2]
    distStr = district[3:5]
    if (distStr == 'AL'):
        distNum = 0
    else: distNum = int(distStr)
    newDist = state + '-' + str(distNum)
    return newDist

In [ ]:
df['District'] = df['District'].apply(lambda x: fixDistricts(x))
df['geoData'] = ''
df.head()

In [ ]:
for row in numpy.arange(0, len(df)-1):
    district = df.iloc[row]["District"]
    geoDF = pd.read_json(requests.get('https://theunitedstates.io/districts/cds/2016/{0}/shape.geojson'.format(district)).text)
    geoData = geoDF.at['coordinates', 'geometry']
    df.iat[row, 4] = geoData[0][0]

In [ ]:
df.head()

In [ ]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [ ]:
db = client.electionDB

In [ ]:
years = ['2016', '2012', '2008']

for year in years:
    for row in numpy.arange(0,len(df)):
        db[year].insert_one({
            "type": "Feature",
            "properties": {
                "District": df.iloc[row]["District"],
                "Result": df.iloc[row][year],
                "Winner": 'R' if df.iloc[row][year] > 50 else 'D'
            },
            "geometry": {
                "type": "polygon",
                "coordinates": [df.iloc[row]["geoData"]]
            }
        })
